In [16]:
from obspy.clients.fdsn import Client
from obspy.core.utcdatetime import UTCDateTime
from obspy import read, read_inventory
from obspy.core import inventory
import numpy as np
import matplotlib as plt
import scipy.io as sio
client = Client("IRIS")
    
t_start = '2020-08-14T10:00:00.0Z'
t_final = '2020-08-14T12:00:00.0Z'

In [36]:
def get_trace(t_start,t_final):
    ## Reads in a start and end time string in the format '2020-08-14T10:00:00.0Z'
    ## Retrieves waveforms as a Stream object from IRIS DMC using ObsPy """
    ## Saves waveform and station metadata and response data in a dictionary """
    ## Saves that dictionary as a .mat file to be accessed in matlab """
    
    ## Internal specifications:
    ## station codes, station locations, station channels, station names

    # Initialize
    location = ['','','','','','','W3','W3','W3','','','','','','','','','']
    stationcode = ['NCHR','NCHR','NCHR','KEMF','KEMF','KEMF','KEMF','KEMF','KEMF','KEMO','KEMO','KEMO','ENWF','ENWF','ENWF','ENHR','ENHR','ENHR']
    # BEFORE NOV 11 2018:
    channel = ['EH*','EH*','CN*','EH*','HH*','HH*',]
    location = ['','','W3','','','',]
    stationcode = ['NCHR','KEMF','KEMF','KEMO','ENWF','ENHR']
    stationname = ['NCHR','KEMF','KEMF_W3','KEMO','ENWF','ENHR']
    # AFTER NOV 11 2018:
    # channel = ['EH*','CN*','EH*','HH*','HH*',]
    # location = ['','W3','','','',]
    # stationcode = ['NCHR','KEMF','KEMO','ENWF','ENHR']
    # stationname = ['NCHR','KEMF_W3','KEMO','ENWF','ENHR']
    # location = ['']
    # stationcode = ['KEMF']
    # channel = ['EH*']
    # stationname = ['KEMF']
    


    t_start = UTCDateTime(t_start)
    t_final = UTCDateTime(t_final)
    
    # AFTER SEPTEMBER 10 2020 (KEMO comes back online with new channels)
    if t_start > UTCDateTime("2020-09-10T00:00:00.0"):
        channel = ['EH*','EH*','CN*','HH*','HH*','HH*',]

    # Loop through stations
    stations = []; sampleRate = []; sampleCount = []; locations = [];
    channels = []; stime = []; etime = []; data = []; networks = [];
    sensitivityFrequency = []; sensitivity = [];
    for i in range(len(channel)):
        try:
            stream = client.get_waveforms(network='NV',station=stationcode[i],location=location[i],channel=channel[i],starttime = t_start,endtime = t_final,attach_response=True)
            for j in range(len(stream)):
                if i==1:
                    master_stream = stream
                else:
                    master_stream = master_stream.append(trace)
                trace = stream[j]
                resp = trace.stats.response._get_overall_sensitivity_and_gain()
                stations.append(trace.stats.station)
                sampleRate.append(trace.stats.sampling_rate)
                sampleCount.append(trace.stats.npts)
                locations.append(trace.stats.location)
                channels.append(trace.stats.channel)
                stime.append(trace.stats.starttime.strftime("%Y-%m-%d:%H:%M:%S.%f"))
                etime.append(trace.stats.endtime.strftime("%Y-%m-%d:%H:%M:%S.%f")) 
                data.append(trace.data)
                networks.append(trace.stats.network)
                sensitivityFrequency.append(resp[0])
                sensitivity.append(resp[1])
                print('Found data for ',stationname[i],' ',trace.stats.channel)
        except:
           print('No data for station',stationname[i])
           continue

    # Save traces
    trace_dict = {'network': networks, 'station': 	stations,'location':locations,'channel':channels,'sensitivity':sensitivity,'sensitivityFrequency':sensitivityFrequency,'data':data,'sampleCount':sampleCount,'sampleRate':sampleRate,'startTime':stime,'endTime':etime}

    return(master_stream)

def save_miniseed(stream,dirname):
    filename = stream[1].stats.network + '_' + stream[1].stats.starttime.strftime("%Y%m%d")
    print(dirname+filename+'.mseed')
    stream.write(dirname+filename+'.mseed', format='MSEED') 


In [29]:
test = get_trace(t_start,t_final)
np.shape(test)

No data for station NCHR
Found data for  KEMF   EHE
Found data for  KEMF   EHN
Found data for  KEMF   EHZ
Found data for  KEMF_W3   CN1
Found data for  KEMF_W3   CN2
Found data for  KEMF_W3   CN3
No data for station KEMO
Found data for  ENWF   HHE
Found data for  ENWF   HHN
Found data for  ENWF   HHZ
No data for station ENHR


/Users/zoekrauss/anaconda3/envs/autolocate/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


(9,)

In [38]:
save_miniseed(test,'test_easyquake/')

Users/zoekrauss/krauss-repo/test_easyquake/NV_20200814.mseed


In [71]:
from easyQuake import download_mseed
from easyQuake import daterange
from datetime import date
from easyQuake import combine_associated
from easyQuake import detection_continuous
from easyQuake import association_continuous

from easyQuake import magnitude_quakeml
from easyQuake import simple_cat_df

import matplotlib.pyplot as plt
maxkm = 300
maxdist=300
lat_a = 47.5
lat_b = 48.5
lon_a = -129.4
lon_b = -128.8


start_date = date(2017, 8, 3)
end_date = date(2017, 8, 30)


project_code = 'endtest'
project_folder = 'test_easyquake'
for single_date in daterange(start_date, end_date):
    print(single_date.strftime("%Y-%m-%d"))
    dirname = single_date.strftime("%Y%m%d")
    # download_mseed(dirname=dirname, project_folder=project_folder, single_date=single_date, minlat=lat_a, maxlat=lat_b, minlon=lon_a, maxlon=lon_b)
    detection_continuous(dirname=dirname, project_folder=project_folder, project_code=project_code, single_date=single_date, machine=True,local=True,machine_picker = 'EQTransformer')
    input('It worked! Press any key')
    association_continuous(dirname=dirname, project_folder=project_folder, project_code=project_code, maxdist=maxdist, maxkm=maxkm, single_date=single_date, local=True)

        
cat, dfs = combine_associated(project_folder=project_folder, project_code=project_code)
cat = magnitude_quakeml(cat=cat, project_folder=project_folder,plot_event=True)
cat.write('catalog_idaho.xml',format='QUAKEML')


catdf = simple_cat_df(cat)
plt.figure()
plt.plot(catdf.index,catdf.magnitude,'.')
    

2017-08-03
KEMO
NCHR
KEMO
NCHR
NCHR
KEMO


FileNotFoundError: [Errno 2] No such file or directory: 'test_easyquake/20170803/gpd_picks.out'

In [70]:
from easyQuake.phasepapy import fbpicker
import os

dir1 = project_folder+'/'+dirname
infile = dir1+'/dayfile.in'
outfile = dir1+'/gpd_picks.out'
pathgpd = '/'.join(str(fbpicker.__file__).split("/")[:-2])+'/gpd_predict'
print(pathgpd + '/gpd_predict.py')
fullpath1 = pathgpd + '/gpd_predict.py'

print(fullpath1+" -V -P -I %s -O %s -F %s" % (infile,outfile, pathgpd))

/Users/zoekrauss/anaconda3/envs/autolocate/lib/python3.6/site-packages/easyQuake/gpd_predict/gpd_predict.py
/Users/zoekrauss/anaconda3/envs/autolocate/lib/python3.6/site-packages/easyQuake/gpd_predict/gpd_predict.py -V -P -I test_easyquake/20170803/dayfile.in -O test_easyquake/20170803/gpd_picks.out -F /Users/zoekrauss/anaconda3/envs/autolocate/lib/python3.6/site-packages/easyQuake/gpd_predict


In [ ]:
detection_continuous(dirname=dirname, project_folder=project_folder, project_code=project_code, single_date=single_date, machine=True,local=True)
    association_continuous(dirname=dirname, project_folder=project_folder, project_code=project_code, maxdist=maxdist, maxkm=maxkm, single_date=single_date, local=True)

cat, dfs = combine_associated(project_folder=project_folder, project_code=project_code)
cat = magnitude_quakeml(cat=cat, project_folder=project_folder,plot_event=True)
cat.write('catalog_idaho.xml',format='QUAKEML')


catdf = simple_cat_df(cat)
plt.figure()
plt.plot(catdf.index,catdf.magnitude,'.')